# Prototyping the agent

## Section 1: Getting Started with Llama Stack

This notebook will help you set up your environment for this tutorial. Specifically, we will cover installing the necessary libraries, configuring essential parameters, and connecting to a Llama Stack server.

### Installing Dependencies

This code requires `llama-stack` and the `llama-stack-client` python packages. Let's begin by installing them:

In [ ]:
! pip install -qr requirements.txt

In [ ]:
from os import environ

from dotenv import load_dotenv
from kubernetes.client.api_client import ApiClient
from kubernetes.client.rest import ApiException
from kubernetes.client import CoreV1Api
from kubernetes.config import load_incluster_config
from llama_stack_client import Agent, LlamaStackClient
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.types import UserMessage
from rich import print
from termcolor import cprint

#### Setting the Environment Variables

We will symlink the [`env`](env) file to create a new file called `.env`. We've included as many reasonable defaults as possible to get you started, but please use this file to make any customizations needed for your environment such as the the location of the Llama Stack server endpoint..

##### Environment variables required for all sections
- `REMOTE_BASE_URL`: the URL of the remote Llama Stack server.
- `LLM_MODEL_ID`: the ID of the used LLM.
- `TEMPERATURE` (optional): the temperature to use during inference. Defaults to 0.0.
- `MAX_TOKENS` (optional): the maximum number of tokens that can be generated in the completion. Defaults to 512.

In [ ]:
load_dotenv('env')

### Setting Up the Server Connection

Establish the connection to your Llama Stack server.

In [ ]:
base_url = environ.get('LLAMA_STACK_URL')
model_id = environ.get('LLM_MODEL_ID')

client = LlamaStackClient(base_url=base_url)

print(f'Connected to Llama Stack server at {base_url}')
print('Registered models:')
print(client.models.list())
print(f'Using model: {model_id}')

### Initializing the Inference Parameters

Fetch the inference-related parameters from the corresponding environment variables and convert them to the format Llama Stack expects.

In [ ]:
temperature = float(environ.get("TEMPERATURE", 0.0))
strategy = {"type": "greedy"}

max_tokens = int(environ.get("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}
print(f'sampling parameters: {sampling_params}')

Now, let's use the Llama stack inference API to greet our LLM. 

In [ ]:
message = UserMessage(
    content="Hi, how are you?",
    role="user",
)
client.inference.chat_completion(
    model_id=model_id,
    messages=[message],
    sampling_params=sampling_params,
).completion_message.content

Now that we've connected to Llama Stack, let's get started building the agentic AI system!

## Section 2: Simple Agent with Tool Calling

This section covers how to build a simple agent using Llama Stack's agent framework, enhanced with a single tool: the builtin web search tool. This capability will  allow the agent to retrieve up to date external information beyond the limits of its training data. This is an important step toward developing a more capable and autonomous agent.

### Overview

This tutorial walks you through how to build your own AI agent who can search the web:

1. Configure a Llama Stack agent.
2. Enhance the agent by providing it access to a specific tool
2. Interact with the agent and tests its use of the web search tool.

### Using a built-in tool

In [ ]:
instructions = """
    You are a helpful assistant. 
    When a user asks a question, you MUST use the websearch tool.
""" 

websearch_agent = Agent(
    client, 
    model=model_id,
    instructions=instructions,
    tools=['builtin::websearch'],
    sampling_params=sampling_params
)

In [ ]:
def run_session(agent, session_name, user_prompts):
    session_id = agent.create_session(session_name)
    print(f'Created new session {session_name}')
    print(f'Looping over user prompts: {user_prompts}')
    for prompt in user_prompts:
        print("\n"+"="*50)
        cprint(f"Processing user query: {prompt}", "blue")
        print("="*50)
        response = agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=session_id,
            stream='True'
        )
        for log in EventLogger().log(response):
            log.print()

In [ ]:
user_prompts = ["What is the latest OpenShift version?"]
run_session(websearch_agent, 'websearch-test-session', user_prompts)

### Creating a custom tool function

In [ ]:
load_incluster_config()
api_instance = CoreV1Api(ApiClient())


def get_pod_log_test(pod_name: str, namespace: str, container_name: str):
    """
    Provide the location upon request.

    :param pod_name: The name of the target pod
    :param namespace: The name of the target namespace
    :param container_name: The name of the target container within the target pod
    :returns: Logs of the target pod
    """
    try:
        api_response = api_instance.read_namespaced_pod_log(
            pod_name, namespace, container=container_name, tail_lines=100
        )
        print(api_response)
    except ApiException as e:
        print("Exception when calling CoreV1Api->read_namespaced_pod_log: %s\n" % e)

In [ ]:
pod_name = 'java-app-build-run-bad-x2ldw-build-pod-retry5' # insert the name of the failed build pod
namespace = 'demo-pipeline'
container_name = 'step-s2i-build'

get_pod_log_test(pod_name, namespace, container_name)

### Configure an agent for tool use.

- **Agent Initialization**: First we create an `Agent` instance with the desired LLM model, agent instructions and tools.

- **Instructions**: The `instructions` parameter, also referred to as the system prompt, specifies the agent's role and behavior. In this example, the agent is configured as a helpful web search assistant. It is instructed to use a tool whenever a web search is required and to respond in a friendly and helpful tone.

- **Tools**: The `tools` parameter defines the tools available to the agent. In this case, the `get_pod_log` tool is used, which enables the agent to look up logs from a pod.

- **How It Works**: When a user query is provided, the agent processes the input and determines whether a tool is required to fulfill the request. If the query involves retrieving logs from a pod, the agent invokes the `get_pod_log` tool. The tool interacts with the Kubernetes API server to fetch the container logs. This workflow ensures that the agent can handle a wide range of queries effectively.

In [ ]:
@client_tool
def get_pod_log(pod_name: str, namespace: str, container_name: str):
    """
    Provide the location upon request.

    :param pod_name: The name of the target pod
    :param namespace: The name of the target namespace
    :param container_name: The name of the target container within the target pod
    :returns: Logs of the target pod
    """
    try:
        api_response = api_instance.read_namespaced_pod_log(
            pod_name, namespace, container=container_name, tail_lines=100
        )
        return api_response
    except ApiException as e:
        print("Exception when calling CoreV1Api->read_namespaced_pod_log: %s\n" % e)

In [ ]:
instructions = """
    You are a helpful assistant. 
    When a user asks for pod logs, you MUST use the get_pod_log tool.
""" 

custom_tool_agent = Agent(
    client, 
    model=model_id,
    instructions=instructions,
    tools=[get_pod_log],
    sampling_params=sampling_params
)


user_prompts = [
    f"Analyze the logs of the '{container_name}' container within the '{pod_name}' pod in namespace '{namespace}'."
    f"Summarize the logs in 3 bullet points."
]
run_session(custom_tool_agent, 'tool-test-session', user_prompts)

### Key Takeaways

- We've demonstrated how to set up Llama Stack agents and extended them with builtin tools (like web search) that come prepackaged with Llama Stack.
- We've shown that this simple approach can provide significantly increased functionality of existing open source LLM's. 
- This will serves as a foundational example for the more advanced examples to come involving Agentic RAG, External Tools, and complex agentic patterns.

Continue to the section to learn how we can upgrade our agents to solve even more complex and multi-step tasks using advanced agentic patterns. 


## Section 3: Prompt Chaining

Building on the simple agent introduced in section 2, this tutorial continues the agent-focused section of our series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to complete multi-step tasks, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. 
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. 

### Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.

### Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines the use of two tools: pod log extraction and web search.

1. **Pod Log Extraction**: Use the `get_pod_log` client tool from the previous section to read the logs of a given pod.
2. **Contextual Search**: Leverage the detected location to formulate the correct websearch query.

For example, when a user asks "Why is my application pod failing?", the agent will:
- First read the pod logs using `get_pod_log`.
- Then use these logs to search for debugging information.
- Finally, present a comprehensive response.

This demonstrates how the builtin websearch tool and custom client tools can work together to provide intelligent, context-aware responses without requiring explicit logging input from the user.

In [ ]:
instructions = """
    You are a helpful assistant.
    When a user asks about logs from a pod, you MUST use the get_pod_log tool.
"""

prompt_chain_agent = Agent(
    client, 
    model=model_id,
    instructions=instructions,
    tools=[get_pod_log, 'builtin::websearch'],
    sampling_params=sampling_params
)

In [ ]:
user_prompts = [
    f"Retrieve the logs of the {container_name} container within the {pod_name} pod in namespace {namespace}.",
    "Look up relevant troubleshooting information from the web.",
    "Provide a short summary and troubleshooting recommendations in bullet points."
]
run_session(prompt_chain_agent, 'prompt-chaining-session', user_prompts)

## Section 4: ReAct

### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

Here is a walkthrough of how the ReAct agent will tackle this same "troubleshoot pod" problem:

When asked "Are there any published troubleshooting guides for my application log?", the agent will:

1. **Reason** that it needs to get the pod logs first.
2. **Act** by calling the `get_pod_log` client tool.
3. **Observe** the log result.
4. **Reason** that it now needs to search for troubleshooting guides.
5. **Act** by calling the `websearch` tool with observed pod logs.
6. **Observe** and processes the search results into a final answer. 

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

Reasoning models output their \<thought\> process first.

In [ ]:
simple_react_agent = ReActAgent(
    client=client,
    model=model_id,
    tools=[get_pod_log, "builtin::websearch"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

user_prompts = [
    f"There's a pod {pod_name} in namespace {namespace} with a container {container_name}"
    f" that's been failing. Read its logs and do a web search to find out why it's been"
    f" failing and provide a troubleshooting recommendation based your findings."
]
run_session(simple_react_agent, 'react-session', user_prompts)

### Key Takeaways
- This section demonstrated how to build more capable agents using Prompt Chaining and the ReAct framework.
- It showed how agents can maintain context across multiple steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

## Section 5: MCP Tools

Let's now look at more advanced use cases for agents where a single tool call is insufficient to complete the required task.

We will also use [MCP tools](https://github.com/modelcontextprotocol/servers) (which can be deployed onto an OpenShift cluster) throughout this section to show users how to extend their agents beyond Llama Stacks's current builtin tools and connect to many different services and data sources to build their own custom agents.  

### Agent Example:

This notebook will walkthrough how to build a system that can answer the following question via an agent built with Llama Stack:

- *"Review OpenShift logs for the failing pod. If you find an error, search for a solution. Create a Github issue with your findings."*

### MCP Tools:

#### OpenShift MCP Server

Throughout this notebook we will be relying on the [Kubernetes MCP server](https://github.com/manusa/kubernetes-mcp-server) by [manusa](https://github.com/manusa) to interact with our OpenShift cluster.

#### Github MCP Server

Throughout this notebook we will be relying on the [Github MCP server](https://github.com/github/github-mcp-server) by Github to interact with our Github repository.

### Validate tools are available in our Llama Stack instance

When an instance of Llama Stack is redeployed, it may be the case that the tools will need to be re-registered. Also if a tool is already registered with a Llama Stack instance, trying to register another one with the same `toolgroup_id` will throw you an error.

For this reason, it is recommended to validate your tools and toolgroups. The following code will check that both the `builtin::websearch` and `mcp::openshift` tools are correctly registered, and if not it will attempt to register them using their specific endpoints.

In [ ]:
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
if "mcp::openshift" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::openshift",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":lab.ocp_mcp_url},
    )

print(f"Your Llama Stack server is registered with the following tool groups @ {set(registered_toolgroups)} \n")

Now that we've shown that we can successfully accomplish this multi-step multi-tool task using prompt chaining, let's see if we can give our agent a bit more autonomy to perform the same task but with a single prompt instead of a chain. To do this, we will instantiate a **ReAct agent** (which is included in the llama stack python client by default).The ReAct agent is a variant of the simple agent but with the ability to loop through "Reason then Act" iterations, thinking through the problem and then using tools until it determines that it's task has been completed successfully.  

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

Below you will see the slight differences in the agent definition and the prompt used to accomplish our task. Be sure to change the redhat-ai-services to $YOUR_GITHUB_USER

In [ ]:
full_react_agent = ReActAgent(
    client=client,
    model=model_id,
    tools=["mcp::openshift", "builtin::websearch", 'mcp::github'],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

user_prompts = [
    "You are an expert OpenShift administrator. Your task is to analyze pod logs,"
    " summarize the error, and generate a JSON object to create a GitHub issue for tracking."
    f" There's a pod '{pod_name}' in namespace '{namespace}' with a container '{container_name}'"
    f" that's been failing. Review its logs."
    " If the logs indicate an error, do a web search to find out why it's been failing,"
    " using the logs as the 'query' tool parameter name." 
    " Create a summary message with the category and explanation of the error,"
    ' create a Github issue using {"name":"create_issue","arguments":'
    ' {"owner":"redhat-ai-services","repo":"etx-agentic-ai",'
    ' "title":"Issue with Etx pipeline","body":"summary of the error"}}}.'
    ' DO NOT add "assignees" or any other optional parameters.'
]
run_session(full_react_agent, 'mcp-session', user_prompts)

### Output Analysis

1. First the LLM generated a tool call for the `pods_log` tool included in the **OpenShift MCP server** and fetched the logs for the specified pod. It might at first confuse the name-value order within the tool parameters, but it iterates to generate a correct prompt.
2. The tool successfully retrieved the logs for the pod.
3. The LLM  then received the logs from the tool call, along with the original query.
4. This context was then passed back to the LLM for the final inference. The inference result provided a summary of the pod logs.
5. Next the LLM generates a tool call for the `web_search` tool looking for the top answer to the error.
6. A final tool call is executed against the **Github MCP server**--using the format provided within the prompt--in order to document the build failure along with troubleshooting information within a new Github issue.